In [25]:
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests

Scraping BBC Using BS4 and HTML Parsing


Scraped On 11/12/2023 4.10 PM


In [80]:
page = 0
url = f"https://www.bbc.co.uk/search?q=Malaysia&d=SEARCH_PS&seqId=062777f0-97ec-11ee-a035-fb9f9909d6da&page={page}"
total_info = []

In [21]:
url = f"https://www.bbc.co.uk/search?q=UK&d=SEARCH_PS&seqId=062777f0-97ec-11ee-a035-fb9f9909d6da&page=1"
webpage = requests.get(url).text
soup = BeautifulSoup(webpage, "html.parser")
max_page = soup.select_one('#main-content > div:nth-child(5) > div > div > nav > div > div > div.ssrcss-1izxn3x-NumberedPagesButtonsContainer.e1ksme8n4 > div > ol > li:nth-child(14) > div > a > div')
print(max_page)

<div class="ssrcss-3vkeha-StyledButtonContent e1kcrsdk1">29</div>


In [ ]:
page = 0
total_info = []
total_articles = 0

while page != 29:
    page += 1
    url = f"https://www.bbc.co.uk/search?q=Malaysia&d=SEARCH_PS&seqId=062777f0-97ec-11ee-a035-fb9f9909d6da&page={page}"
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, "html.parser")
    print(f'Printing {page}')   
    for article in soup.find_all('div',class_='ssrcss-tq7xfh-PromoContent exn3ah99'):
        total_articles += 1
        title = article.find('span').text #get article titles
        description = article.find('p',class_='ssrcss-1q0x1qg-Paragraph e1jhz7w10').text #get article descs
        misc_list = article.find('ul',class_='ssrcss-1xpwu3-MetadataStripContainer eh44mf03') # get miscellanous info such as date published, section and area

        for list_item in misc_list.findAll('li'):
            if list_item.text.startswith('Published'):
                time = list_item.text.split('Published')[1]
            if list_item.text.startswith('Site'):
                site = list_item.text.split('Site')[1]
            if list_item.text.startswith('Section'):
                section = list_item.text.split('Section')[1]

        row_info = [title,description,time,site,section]
        total_info.append(row_info)  
   
print(f'Total Articles : {total_articles}')
 

Printing 1
Printing 2
Printing 3
Printing 4
Printing 5
Printing 6
Printing 7
Printing 8
Printing 9
Printing 10
Printing 11
Printing 12
Printing 13
Printing 14
Printing 15
Printing 16
Printing 17
Printing 18
Printing 19
Printing 20
Printing 21
Printing 22
Printing 23
Printing 24
Printing 25
Printing 26
Printing 27
Printing 28
Printing 29
Total Articles : 290


In [36]:
def time_processor(date):
    today = datetime.today().date()
    if date.endswith('ago'):
        if 'hour' in date:
           date = today.strftime('%d %B %Y')
        if 'days' in date:
           date = (today - timedelta(days=int(date.split(' ')[0]))).strftime('%d %B %Y')
    elif date[len(date)-4:].isnumeric() == False:
           date = date + ' 2023'
    return date

In [37]:
def data_pipeline(info):
    #Make DF
    df = pd.DataFrame(info,columns=['title','description','publisheddate','section','site'])

    #Convert Written Dates to Datetime OBJ
    print(df['publisheddate'])
    df['publisheddate'] = df['publisheddate'].apply(lambda x : time_processor(x))
    print(df['publisheddate'])
    df['publisheddate'] = pd.to_datetime(df['publisheddate'])
    df['publisheddate'] = df.sort_values(by='publisheddate',ascending=False).reset_index(drop=True)
    print(df)

In [38]:
data_pipeline(total_info)

0       10 December
1        8 December
2        4 December
3       13 November
4       12 November
           ...     
285    27 July 2017
286    20 July 2017
287     5 July 2017
288     5 July 2017
289    30 June 2017
Name: publisheddate, Length: 290, dtype: object
0      None
1      None
2      None
3      None
4      None
       ... 
285    None
286    None
287    None
288    None
289    None
Name: publisheddate, Length: 290, dtype: object


ValueError: Columns must be same length as key

In [104]:
df = pd.DataFrame(total_info,columns=['title','description','publisheddate','section','site'])
df

,title,description,publisheddate,section,site
0,The $100bn ghost city in southern Malaysia,The Chinese-built city was supposed to be a th...,15 hours ago,News,Asia
1,Snoop Dogg & an ex-Wolves midfielder - meet Ma...,Former Wolves midfielder Hong Wan has moved fr...,2 days ago,Sport,Football
2,Forest City: Inside Malaysia's Chinese-built '...,"Built during the Chinese property boom, Forest...",6 days ago,News,Business
3,Malaysian killer released from Australian immi...,An ex-policeman convicted in a politically cha...,13 November,News,Australia
4,MotoGP: Enea Bastianini claims first race win ...,Ducati's Enea Bastianini claims his first race...,12 November,Sport,Motorsport
...,...,...,...,...,...
285,Najib Razak: Malaysian ex-PM gets 12-year jail...,Najib Razak gets 12 years in jail after he is ...,28 July 2020,News,Asia
286,Newsday: Guilty verdict for former Malaysian p...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
287,Newsday: Former Malaysian PM guilty of abuse o...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
288,North Korea v Malaysia: Asian Cup qualifier is...,"""Geo-political tension"" and ""safety and securi...",17 May 2017,Sport,Football


In [105]:
df.to_csv('BBCNews.csv',index=False)